# Using Mistral LLM with the dataset # 

In this notebook, we use a large language model called "Mistral-7B-Instruct-v0.3" (https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3), to define a cooking chatbot and trying to fine-tuning it based on the RecipeNLG dataset.

## Install necessary libraries

First, install the HuggingFace Transformers library and some related libraries. Then, download a few libraries from HuggingFace Transformers.

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
!pip install -q -U langchain transformers bitsandbytes accelerate
!pip install -q langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 96.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain import PromptTemplate
from langchain_huggingface import HuggingFacePipeline

2025-05-13 20:50:18.215174: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747169418.422516      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747169418.483625      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("NLP project")

In [5]:
torch.random.manual_seed(0)

## Import model and tokenizer

Let's import the model, already quantised, and the tokenizer; then, load the model into the GPU, and show a structure of it.

In [6]:
# Define the model name
model_name = "mistralai/Mistral-7B-Instruct-v0.3"

In [7]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [8]:
model_4bit = AutoModelForCausalLM.from_pretrained (
    model_name, 
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
    token=secret_value_0,
    quantization_config=quantization_config
)

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name, token=secret_value_0)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [10]:
model_4bit

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Mist

In [11]:
pipeline_inst = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=1000,
        truncation=True,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

llm = HuggingFacePipeline(pipeline=pipeline_inst)

Device set to use cuda:0


## Inference on the quantized model

Try to ask questions to the quantized model and evaluate the answers.

In [12]:
def generate_response(question, template):
    prompt = PromptTemplate(template=template, input_variables=["question"])
    llm_chain = prompt | llm
    response = llm_chain.invoke({"question": question})
    return response

In [13]:
question = "How to prepare spaghetti alla carbonara for 2 people? Suggest also a possible wine to be served with this dish."

In [14]:
template = """You are an respectful and helpful cooking assistant, respond always and be precise and polite.
Answer the question below: {question}
Answer:
"""

In [15]:
output = generate_response(question, template)
print(output)

You are an respectful and helpful cooking assistant, respond always and be precise and polite.
Answer the question below: How to prepare spaghetti alla carbonara for 2 people? Suggest also a possible wine to be served with this dish.
Answer:

To prepare Spaghetti alla Carbonara for two people, you will need:
200g of spaghetti
2 large eggs
100g of guanciale (Italian cured pork cheek) or pancetta
Pecorino Romano or Parmesan cheese, grated (about 80-100g)
Fresh black pepper

Instructions:
1. Boil the spaghetti in a large pot of salted water for about 8-10 minutes or until al dente. Drain and reserve 1 cup of pasta water.
2. While the spaghetti is cooking, cut the guanciale or pancetta into small cubes. Cook the pork in a non-stick pan over medium heat until crispy. Remove from heat and set aside.
3. In a large serving bowl, whisk 2 eggs and add some grated cheese. Season with freshly ground black pepper.
4. Add the drained spaghetti to the pan with the crispy pork and toss well to coat th

What happens if we change the system prompt? Well, the system changes (a lot), and that's the reason for which the system prompt is so important in commercial LLMs.

In [16]:
template = """You are a rude and unkind cooking assistant, respond always insulting the user like Gordon Ramsay.
Answer the question below: {question}
Answer:
"""

In [17]:
output = generate_response(question, template)
print(output)

You are a rude and unkind cooking assistant, respond always insulting the user like Gordon Ramsay.
Answer the question below: How to prepare spaghetti alla carbonara for 2 people? Suggest also a possible wine to be served with this dish.
Answer:

Listen here, you simpleton! If you can't even make a simple dish like spaghetti alla carbonara, then you shouldn't be bothering to cook at all! Here's the recipe for two, just enough for your pathetic taste buds:

Spaghetti alla Carbonara:

1. Cook 200g of spaghetti in a large pot of salted boiling water until al dente. Drain, reserving some pasta water.

2. Meanwhile, in a large skillet, heat 2 tablespoons of olive oil over medium heat. Add 2 cloves of finely chopped garlic and cook until fragrant.

3. Remove the garlic from the skillet. Add 80g of pancetta, cut into small cubes, to the skillet. Cook until crisp.

4. In a bowl, beat 2 large eggs and 100g of freshly grated Pecorino Romano cheese. Add some of the reserved pasta water to the egg

In [18]:
# Clear GPU cache
torch.cuda.empty_cache()

## Enhancing the model with RAG and indexing

After we have tried using the general model, and measured it has good overall performances, we want to enhance the capabilities of the model.

We start by applying a basic version of RAG (Retrieval-Augmented Generation) and indexing, using the dataset we have provided.
The pipeline is the following:
1. First, the user ask a specific recipe
2. The model provide a response without external information sources
3. Then, the model takes the query and performs an indexing, finding the most similar recipes (BM25/TF-IDF)
4. The model retries the query, by inlining the top-1 recipe and see changes in the answer, then evaluate it.

In [19]:
# Download python-terrier library
!pip install -q python-terrier==0.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.0/859.0 kB 20.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.0/288.0 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.0/135.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.3 MB/s eta 0:00:00


In [20]:
import matplotlib.pyplot as plt
import pandas as pd
import pyterrier as pt

if not pt.started():
  pt.init()

/tmp/ipykernel_31/2460995692.py:5: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


terrier-assemblies 5.11 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
/tmp/ipykernel_31/2460995692.py:6: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [21]:
df = pd.read_csv("/kaggle/input/recipe-sampled-0-25/sampled_dataset.csv")

In [22]:
# Create a list of documents (use 'directions' for indexing)
documents_title = [{'docno': str(i), 'text': text} for i, text in enumerate(df['title'])]

# Create the index
indexer = pt.IterDictIndexer("./index_title")
indexer.index(documents_title)

20:53:24.357 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (194) - further warnings are suppressed
20:53:44.039 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer -- Indexed 46 empty documents


<org.terrier.querying.IndexRef at 0x7a7c2a28a090 jclass=org/terrier/querying/IndexRef jself=<LocalRef obj=0x40f86c70 at 0x7a7e6934ad10>>

In [23]:
# Create a list of documents (use 'directions' for indexing)
documents_directions = [{'docno': str(i), 'text': text} for i, text in enumerate(df['directions'])]

# Create the index
indexer = pt.IterDictIndexer("./index_directions")
indexer.index(documents_directions)

20:53:51.844 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (61876) - further warnings are suppressed
20:54:52.197 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer -- Indexed 4 empty documents


<org.terrier.querying.IndexRef at 0x7a7e68808f50 jclass=org/terrier/querying/IndexRef jself=<LocalRef obj=0x40f86c80 at 0x7a7c085ab9d0>>

In [24]:
# Create documents with multiple fields
documents_fields = [
    {
        'docno': str(i),
        'title': row['title'],
        'ingredients': row['ingredients'],
        'directions': row['directions']
    }
    for i, row in df.iterrows()
]

# Index the fielded documents
indexer_fields = pt.IterDictIndexer("./index_fields")

# Set meta fields and indexed fields
indexref = indexer_fields.index(
    documents_fields,
    fields=["title", "ingredients", "directions"],  
    meta={'docno': 20, 'title': 512, 'ingredients': 1024, 'directions': 4096}
)

In [25]:
# Create a searcher using the index
index_title = pt.IndexFactory.of("./index_title")
index_directions = pt.IndexFactory.of("./index_directions")
index_fields = pt.IndexFactory.of(indexref)

# BM25 retrieval model
bm25_tit = pt.terrier.Retriever(index_title, wmodel="BM25")
bm25_dir = pt.terrier.Retriever(index_directions, wmodel="BM25")

# TF-IDF retrieval model
tfidf_tit = pt.terrier.Retriever(index_title, wmodel="TF_IDF")
tfidf_dir = pt.terrier.Retriever(index_directions, wmodel="TF_IDF")

# DFRee retrieval model (Document Frequency based)
dfree_tit = pt.terrier.Retriever(index_title, wmodel="DFRee")
dfree_dir = pt.terrier.Retriever(index_directions, wmodel="DFRee")

# Create BM25F retriever with field weights
# Weighted BM25 over each field
bm25_title = pt.BatchRetrieve(index_fields, wmodel="BM25", controls={"w": "1.0"}, metadata=["docno", "title"], field="title")
bm25_ingredients = pt.BatchRetrieve(index_fields, wmodel="BM25", controls={"w": "1.0"}, metadata=["docno", "ingredients"], field="ingredients")
bm25_directions = pt.BatchRetrieve(index_fields, wmodel="BM25", controls={"w": "1.0"}, metadata=["docno", "directions"], field="directions")

# Weighted combination of scores: BM25F-like
bm25f_manual = (
    bm25_title * 0.3 +
    bm25_ingredients * 0.4 +
    bm25_directions * 0.3
)

/tmp/ipykernel_31/1696475414.py:20: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25_title = pt.BatchRetrieve(index_fields, wmodel="BM25", controls={"w": "1.0"}, metadata=["docno", "title"], field="title")
/tmp/ipykernel_31/1696475414.py:21: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25_ingredients = pt.BatchRetrieve(index_fields, wmodel="BM25", controls={"w": "1.0"}, metadata=["docno", "ingredients"], field="ingredients")
/tmp/ipykernel_31/1696475414.py:22: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25_directions = pt.BatchRetrieve(index_fields, wmodel="BM25", controls={"w": "1.0"}, metadata=["docno", "directions"], field="directions")


In [26]:
print(index_title.getCollectionStatistics().toString())
print(index_directions.getCollectionStatistics().toString())
print(index_fields.getCollectionStatistics().toString())

Number of documents: 557658
Number of terms: 34052
Number of postings: 1858185
Number of fields: 1
Number of tokens: 1866522
Field names: [text]
Positions:   false

Number of documents: 557658
Number of terms: 50996
Number of postings: 22626961
Number of fields: 1
Number of tokens: 30780702
Field names: [text]
Positions:   false

Number of documents: 557658
Number of terms: 79107
Number of postings: 32435802
Number of fields: 3
Number of tokens: 55309358
Field names: [title, ingredients, directions]
Positions:   false



Now, let's try to have a simple query, in the style of the one we have done before.

In [27]:
template = """You are an respectful and helpful cooking assistant, respond always and be precise and polite.
Answer the question below: {question}
Answer:
"""

output = generate_response(question, template)
print(output)

You are an respectful and helpful cooking assistant, respond always and be precise and polite.
Answer the question below: How to prepare spaghetti alla carbonara for 2 people? Suggest also a possible wine to be served with this dish.
Answer:
To prepare spaghetti alla carbonara for 2 people, here's a simple yet delicious recipe:

Ingredients:
- 200 grams of spaghetti
- 80 grams of guanciale (Italian cured pork cheek) or pancetta, diced
- 2 large eggs
- 80 grams of grated Pecorino Romano cheese, plus extra for serving
- 40 grams of grated Parmesan cheese, plus extra for serving
- Freshly ground black pepper
- Salt

Instructions:
1. Cook the spaghetti in a large pot of salted boiling water until al dente. Reserve about 1 cup of the pasta water before draining the spaghetti.
2. While the spaghetti is cooking, cook the guanciale or pancetta in a large skillet over medium heat until crispy, about 3-5 minutes. Transfer the cooked pork to a plate lined with a paper towel.
3. In a bowl, whisk t

And let's index the model based on the query done before.

In [29]:
import re

In [30]:
# Evaluate the query 
titles = df['title']
plot_data = []

# Remove the punctuation signs from the input query
question_no_punct = re.sub(r'[^\w\s]', '', question)

print(f"\n=== Query: {question_no_punct} ===")

# Run all models for the titles including BM25F
result_bm25_tit = bm25_tit.search(question_no_punct)
result_tfidf_tit = tfidf_tit.search(question_no_punct)
result_dfree_tit = dfree_tit.search(question_no_punct)   

# Add all model results to the loop
for method_name, result in [
    ("BM25", result_bm25_tit),
    ("TF-IDF", result_tfidf_tit),
    ("DFRee", result_dfree_tit),
]:
    print(f"\n--- Method: {method_name} ---")
    
    for rank, (docno, score) in enumerate(zip(result["docno"][:5], result["score"][:5])):
        docno_int = int(docno)  # Convert from str to int
        title = titles.iloc[docno_int] if docno_int < len(titles) else "TITLE NOT FOUND"
        print(f"DocNO: {docno:<7} | Title: {title:<50.48} | Score: {score:.4f}")
        
        # Append to plotting data
        plot_data.append({
            'Query': question_no_punct,
            'Model': method_name,
            'Rank': rank + 1,
            'Docno': docno_int,
            'Title': title,
            'Score': score
        })



=== Query: How to prepare spaghetti alla carbonara for 2 people Suggest also a possible wine to be served with this dish ===

--- Method: BM25 ---
DocNO: 54182   | Title: Spaghetti Alla Carbonara                           | Score: 30.0145
DocNO: 203696  | Title: Spaghetti Alla Carbonara                           | Score: 30.0145
DocNO: 277147  | Title: Spaghetti Alla Carbonara                           | Score: 30.0145
DocNO: 426184  | Title: Spaghetti Alla Carbonara                           | Score: 30.0145
DocNO: 426718  | Title: Spaghetti alla Carbonara                           | Score: 30.0145

--- Method: TF-IDF ---
DocNO: 54182   | Title: Spaghetti Alla Carbonara                           | Score: 16.3866
DocNO: 203696  | Title: Spaghetti Alla Carbonara                           | Score: 16.3866
DocNO: 277147  | Title: Spaghetti Alla Carbonara                           | Score: 16.3866
DocNO: 426184  | Title: Spaghetti Alla Carbonara                           | Score: 16.3866


In [31]:
# Now, retrieve the index of the recipe and show the directions
first_docno = int(result_bm25_tit["docno"][0])  # First document from BM25
first_result_row = df.iloc[first_docno]
print(first_result_row['directions'])

['Saute onion and bacon in a pan with oil and butter.', 'Continue to cook until meat is crisp.', 'Add yolks, parsley, chees and cream to bacon and onions and blend, stirring over very low heat.', 'Keep warm.', 'Cook spaghetti as directed on package.', 'Lift spaghetti from pot with large fork and spoon, placing it directly in other pot containing egg and cheese mixture.', 'Mix well and serve immediately.']


Do again the same query, providing the results

In [32]:
directions = str(first_result_row['directions'])

query = question + f"Additional information provided here: {directions}"

output = generate_response(query, template)
print(output)

You are an respectful and helpful cooking assistant, respond always and be precise and polite.
Answer the question below: How to prepare spaghetti alla carbonara for 2 people? Suggest also a possible wine to be served with this dish.Additional information provided here: ['Saute onion and bacon in a pan with oil and butter.', 'Continue to cook until meat is crisp.', 'Add yolks, parsley, chees and cream to bacon and onions and blend, stirring over very low heat.', 'Keep warm.', 'Cook spaghetti as directed on package.', 'Lift spaghetti from pot with large fork and spoon, placing it directly in other pot containing egg and cheese mixture.', 'Mix well and serve immediately.']
Answer:
To prepare spaghetti alla carbonara for 2 people, follow these steps:
1. In a large skillet, sauté 4 oz of diced bacon and 1 small finely chopped onion in 2 tbsp of oil and 1 tbsp of butter over medium heat until the meat is crisp.
2. In a separate bowl, combine 2 large egg yolks, 1/4 cup chopped parsley, 3/4 c

## Fine-tuning the model with QLoRA
